In [1]:
import numpy as np
import pandas as pd
import re

In [34]:
places = pd.read_csv('Review_db.csv')

In [35]:
places.shape

(1482466, 7)

In [36]:
places.head()

,City,Place,Review,Rating,Name,Date,Raw_Review
0,Aamby Valley City,19 Degree North,aamby valley beautiful place clear blue skies ...,5,Anonymous,NaN,Aamby valley is a beautiful place with its cle...
1,Aamby Valley City,19 Degree North,executed obt akshay thanx team thoroughly enjo...,4,Anonymous,NaN,Very well executed obt by Akshay.... Thanx as ...
2,Aamby Valley City,19 Degree North,awesome experience atv tracts obstacles mainta...,5,Anonymous,NaN,Awesome experience at the ATV\nTracts and obst...
3,Aamby Valley City,19 Degree North,visited aamby valley yesterday short excursion...,4,Anonymous,NaN,we visited the Aamby Valley yesterday for shor...
4,Aamby Valley City,19 Degree North,far mumbai place finest adventure places visit...,5,Anonymous,NaN,"Not far from Mumbai, this place is one of the ..."


In [37]:
places.duplicated().sum()

14220

# Data Pre-Processing 

In [38]:
places = places.drop_duplicates()

In [39]:
places.drop(columns='Name', inplace=True)

In [40]:
places.drop(columns='Date', inplace=True)

In [41]:
places.duplicated(subset=['City', 'Place']).sum()

1453007

In [42]:
places[places['City']=='Aamby Valley City']

,City,Place,Review,Rating,Raw_Review
0,Aamby Valley City,19 Degree North,aamby valley beautiful place clear blue skies ...,5,Aamby valley is a beautiful place with its cle...
1,Aamby Valley City,19 Degree North,executed obt akshay thanx team thoroughly enjo...,4,Very well executed obt by Akshay.... Thanx as ...
2,Aamby Valley City,19 Degree North,awesome experience atv tracts obstacles mainta...,5,Awesome experience at the ATV\nTracts and obst...
3,Aamby Valley City,19 Degree North,visited aamby valley yesterday short excursion...,4,we visited the Aamby Valley yesterday for shor...
4,Aamby Valley City,19 Degree North,far mumbai place finest adventure places visit...,5,"Not far from Mumbai, this place is one of the ..."
5,Aamby Valley City,19 Degree North,yearn adrenaline rush place try atv forester m...,5,if you yearn for the adrenaline rush then this...
6,Aamby Valley City,19 Degree North,great adventure place recommend x jungle safar...,5,A great adventure place .I recommend there 4x...
98,Aamby Valley City,19 Degree North,visited adventitious sports atv biking zip lin...,4,Visited here for adventitious sports. we did a...
100,Aamby Valley City,19 Degree North,stayed property time stayed aamby valley visit...,3,I haven't stayed at this property but every-t...
102,Aamby Valley City,19 Degree North,went couple friends weekend amazing played pai...,5,We went there with couple of our friends last ...


In [43]:
places['Rating'].max()

5

In [44]:
places[places['City'] == 'Aamby Valley City']['Raw_Review'][0]

"Aamby valley is a beautiful place with its clear blue skies and fresh green grass. My family and I visited aamby valley to celebrate my mother's birthday. My mom had the most splendid time. Thanks to Pinky Bharadwaj for handling our booking from Bombay. Together..."

In [45]:
places[places['City'] == 'Aamby Valley City']['Review'][0]

'aamby valley beautiful place clear blue skies fresh green grass family visited aamby valley celebrate mother birthday mom splendid time thanks pinky bharadwaj handling booking bombay'

In [46]:
places.drop(columns='Raw_Review', inplace=True)

In [47]:
places

,City,Place,Review,Rating
0,Aamby Valley City,19 Degree North,aamby valley beautiful place clear blue skies ...,5
1,Aamby Valley City,19 Degree North,executed obt akshay thanx team thoroughly enjo...,4
2,Aamby Valley City,19 Degree North,awesome experience atv tracts obstacles mainta...,5
3,Aamby Valley City,19 Degree North,visited aamby valley yesterday short excursion...,4
4,Aamby Valley City,19 Degree North,far mumbai place finest adventure places visit...,5
...,...,...,...,...
1482461,Zuluk,Zuluk Wildlife Area,zuluk week march mst virgin unexplored region ...,3
1482462,Zuluk,Zuluk Wildlife Area,planned visit eastern sikkim time covered stat...,3
1482463,Zuluk,Zuluk Wildlife Area,excellent watched place east sikkim visited pl...,5
1482464,Zuluk,Zuluk Wildlife Area,beautiful areas sikkim falls eastern sikkim an...,4


In [48]:
new_places = pd.DataFrame()

In [49]:
places[places['City'] == 'Kanpur']['Place'].unique()

array(['Allen Forest Zoo', 'A to Z Shopping Mall', 'Bhitargaon Temple',
       'Blue World Theme Park', 'Anandeshwar Temple', 'Brahmavart Ghat',
       'Dashanan Mandir', 'Green Park Stadium', 'Dwarkadhish Temple',
       'Dhruv Teela', 'Harcourt Butler Technological Institute',
       'ISKCON Kanpur, Sri Sri Radha Madhav Temple', 'Habiba Masjid',
       'Jain Glass Temple', 'Jama Masjid', 'Jagannath Temple',
       'Kamla Retreat', 'Kanpur Memorial Church',
       'Kanpur Sangrahalaya (Kanpur Museum)',
       'Massacre Ghat (Satti Chaura Ghat)', 'Moti Jheel', 'Phool Bagh',
       'Nana Rao Smarak Park', 'Rave 3 Mall', 'Shri Radhakrishna Temple',
       'Sai Mandir', 'Valmiki Ashram Temple', 'Sisamau Bazaar',
       'Som Dutt Plaza', 'Tapeshwari Devi Temple'], dtype=object)

In [50]:
new_places[['City', 'Place']] = places[['City', 'Place']].drop_duplicates()

In [51]:
new_places

,City,Place
0,Aamby Valley City,19 Degree North
7,Abhaneri,Chand Baori (Step well)
716,Abhaneri,Harshat Mata Temple
834,Achooranam,Wayanad Tea Museum
865,Acharapakkam,Mazhai Malai Madha Shrine
...,...,...
1482341,Ziro,Pine Grove
1482347,Zuluk,Zuluk Wildlife Area
1482357,Zuluk,Thambi View Point
1482361,Ziro,Tarin Fish Farm


In [52]:
new_places.values

array([['Aamby Valley City', '19 Degree North'],
       ['Abhaneri', 'Chand Baori (Step well)'],
       ['Abhaneri', 'Harshat Mata Temple'],
       ...,
       ['Zuluk', 'Thambi View Point'],
       ['Ziro', 'Tarin Fish Farm'],
       ['Ziro', 'Talley Valley Wildlife Sanctuary']], dtype=object)

In [54]:
places[places['City']=='Agra'][places['Place']=='Taj Mahal']['Rating'].count()

C:\Users\HP\AppData\Local\Temp\ipykernel_12648\282328005.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  places[places['City']=='Agra'][places['Place']=='Taj Mahal']['Rating'].count()


23215

In [86]:
new_places[new_places['City']=='Agra'][new_places['Place'] == 'Taj Mahal']['Review'].values[0]

C:\Users\HP\AppData\Local\Temp\ipykernel_12648\3253607850.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_places[new_places['City']=='Agra'][new_places['Place'] == 'Taj Mahal']['Review'].values[0]


'went early crowds bad paid guide raja easy experience magnificent building indian ticked bucket listsuggest best timid group visit place sunrise hire guide photographer explain best shots wonder worldhope heavy segregated male female security inside ok food organised spend long really soakabsolute travelling northern india purely breathtaking great guide accompany gain appreciation history surrounds great wonder worldstar truly amazing experience person went early beat crowds important reminder bring selfie stick tripod allowedromantic story romantic place building massive inlay semi precious gems taken years complete aware restricted water small bag cameras mobile allowedtime place agra disappointed taj mahal ensure guide maximise visitsaid taj mahal history cultural impact came agra self organised golden triangle tour specifically taj shop driver guide recommendedglimpse goosebumps great monument individuals best buy ticket online avoid queues beware guides photographers storm easy 

In [91]:
text = new_places[new_places['Place'] == 'Safed Akhara']['Review'].values[0]
text

'य प र च न क ल म आस थ क क द र बन ह आ ह और यह भ कह ज त ह क इस जगह पर sache द ल स द आ म ग त व जर र प र ह त ह famous temple barmer km barmer templetemples shiv ji saraswati ji shri radha krishan shri laxmi ganesh hanuman ji temple pyau drinking water lucky peacock try bring food birdssafed akhara located way mahabar sand dunes approximately km km barmer visit temple continue way mahabar sand dunes temple various shrines dedicated lord shiva radhesafed aakra palace god lord shiva kilometer barmer city famous mhabaar sand dunes located nearly'

In [96]:
text1 = ''.join([i if ord(i) < 128 else '' for i in text])
text1

'                             sache                 famous temple barmer km barmer templetemples shiv ji saraswati ji shri radha krishan shri laxmi ganesh hanuman ji temple pyau drinking water lucky peacock try bring food birdssafed akhara located way mahabar sand dunes approximately km km barmer visit temple continue way mahabar sand dunes temple various shrines dedicated lord shiva radhesafed aakra palace god lord shiva kilometer barmer city famous mhabaar sand dunes located nearly'

In [102]:
re.sub(' +',' ',text1)

' sache famous temple barmer km barmer templetemples shiv ji saraswati ji shri radha krishan shri laxmi ganesh hanuman ji temple pyau drinking water lucky peacock try bring food birdssafed akhara located way mahabar sand dunes approximately km km barmer visit temple continue way mahabar sand dunes temple various shrines dedicated lord shiva radhesafed aakra palace god lord shiva kilometer barmer city famous mhabaar sand dunes located nearly'

In [58]:
rating = []
for val in new_places.values:
    rating.append(places[places['City']==val[0]][places['Place']==val[1]]['Rating'].mean())

C:\Users\HP\AppData\Local\Temp\ipykernel_12648\3143151218.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rating.append(places[places['City']==val[0]][places['Place']==val[1]]['Rating'].mean())


In [59]:
review = []
for val in new_places.values:
    combine = lambda x: "".join(x)
    review.append(combine(places[places['City']==val[0]][places['Place']==val[1]]['Review'].values))

C:\Users\HP\AppData\Local\Temp\ipykernel_12648\1339785499.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  review.append(combine(places[places['City']==val[0]][places['Place']==val[1]]['Review'].values))


In [55]:
count = []
for val in new_places.values:
    count.append(places[places['City']==val[0]][places['Place']==val[1]]['Rating'].count())

C:\Users\HP\AppData\Local\Temp\ipykernel_12648\3313658952.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  count.append(places[places['City']==val[0]][places['Place']==val[1]]['Rating'].count())


In [60]:
new_places['Review'] = review

In [61]:
new_places['Rating'] = rating

In [62]:
new_places['Count'] = count

<h2>Function to remove non ascii character from text</h2>

In [104]:
def remove_non_ascii(text):
    text1 = ''.join([i if ord(i) < 128 else '' for i in text])
    return re.sub(' +',' ',text1)

In [106]:
new_places['Review']=new_places['Review'].apply(lambda x: remove_non_ascii(x))

In [90]:
new_places.to_csv('places.csv', header=True, index=True)

In [112]:
new_places.shape

(15239, 5)

In [113]:
new_places['Review'][0]

'aamby valley beautiful place clear blue skies fresh green grass family visited aamby valley celebrate mother birthday mom splendid time thanks pinky bharadwaj handling booking bombayexecuted obt akshay thanx team thoroughly enjoyed especially frame treasure huntawesome experience atv tracts obstacles maintained safe challenging blast good experience helpful instructorsvisited aamby valley yesterday short excursion trip mumbai drove travel time approx hours city clean away polluted air mumbai lunch woodpecker hotel truly impressedfar mumbai place finest adventure places visited recent times worth going safety standards maintained enjoyyearn adrenaline rush place try atv forester mins rush loads stuff including kiddie zip line liked beautiful mountain backdrop whizzgreat adventure place recommend x jungle safari special thanks mr rajesh offers zorbing rappelling wall climbing paintball following international safety standards equipments maintainedvisited adventitious sports atv biking z

In [114]:
# remove repetitive words
new_places['Review'] = new_places['Review'].apply(lambda x:' '.join(set(x.split())))

In [115]:
new_places['Review'][0]

'requested lining green couple international try mr centre road track maintained truly adventure safe enjoyyearn north line whizzgreat loads gate reaching short played friends paint helpful thanx experience instructorsvisited going adventitious weekend rock th office building amby travel sure destination party fresh including booking yesterday x things engaging kiddie ride worth frame rajesh wanted kinds amazing vehicle miss polite backdrop mountain excursion places valley ball team visit sky room friendly times good gazer grass jungle aamby property biking trip finest mom worthdegree helped following visited beautiful mtstook obstacles exit witness activities cruises quite fun splendid special climbing hours recent thoroughly blast place adrenaline indiaamazing long cake zip safari hotel family visitingstayed darkness experienced clean son executed zipline especially handling degree time people located equipments events pinky safety stuff bus organised polluted really restaurantexcell

In [14]:
total_review = ''
for str in df['Review'].values:
    total_review += str

In [15]:
len(total_review)

33443484

In [16]:
len(set(total_review.split()))

549982

In [116]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [117]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [118]:
new_places['Review'] = new_places['Review'].apply(stem)

In [119]:
new_places['Review'][0]

'request line green coupl intern tri mr centr road track maintain truli adventur safe enjoyyearn north line whizzgreat load gate reach short play friend paint help thanx experi instructorsvisit go adventiti weekend rock th offic build ambi travel sure destin parti fresh includ book yesterday x thing engag kiddi ride worth frame rajesh want kind amaz vehicl miss polit backdrop mountain excurs place valley ball team visit sky room friendli time good gazer grass jungl aambi properti bike trip finest mom worthdegre help follow visit beauti mtstook obstacl exit wit activ cruis quit fun splendid special climb hour recent thoroughli blast place adrenalin indiaamaz long cake zip safari hotel famili visitingstay dark experienc clean son execut ziplin especi handl degre time peopl locat equip event pinki safeti stuff bu organis pollut realli restaurantexcel dj approx sky recommend drove rush park distanc like sport rock experienceadventur huntawesom obt standard min paintbal cottag challeng priv

<h2>Function to remove underscore from the string</h2>

In [168]:
def remove_underscore(text):
    return text.replace('_', ' ')

In [169]:
new_places['Review'] = new_places['Review'].apply(remove_underscore)

In [170]:
new_places.to_csv('places.csv', header=True, index=False)

In [2]:
df = pd.read_csv('places.csv')

In [4]:
df.head()

,City,Place,Review,Rating,Count
0,Aamby Valley City,19 Degree North,request line green coupl intern tri mr centr r...,4.588235,17
1,Abhaneri,Chand Baori (Step well),older likevillag green blew store region year ...,4.248996,747
2,Abhaneri,Harshat Mata Temple,sudden blew chond road tri affluenc renov agra...,3.915254,118
3,Achooranam,Wayanad Tea Museum,roof green musium tri mr museumhi year floorfe...,4.136364,44
4,Acharapakkam,Mazhai Malai Madha Shrine,nadu shrine follow goe good tamil car interced...,5.000000,1


In [5]:
df[df['City']=='Agra'][df['Place']=='Taj Mahal']

C:\Users\HP\AppData\Local\Temp\ipykernel_22556\2235406759.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['City']=='Agra'][df['Place']=='Taj Mahal']


,City,Place,Review,Rating,Count
151,Agra,Taj Mahal,com agentsup advis region krishnatruli terrif ...,4.322593,23215


In [5]:
np.array(df['Review'].values)

array(['request line green coupl intern tri mr centr road track maintain truli adventur safe enjoyyearn north line whizzgreat load gate reach short play friend paint help thanx experi instructorsvisit go adventiti weekend rock th offic build ambi travel sure destin parti fresh includ book yesterday x thing engag kiddi ride worth frame rajesh want kind amaz vehicl miss polit backdrop mountain excurs place valley ball team visit sky room friendli time good gazer grass jungl aambi properti bike trip finest mom worthdegre help follow visit beauti mtstook obstacl exit wit activ cruis quit fun splendid special climb hour recent thoroughli blast place adrenalin indiaamaz long cake zip safari hotel famili visitingstay dark experienc clean son execut ziplin especi handl degre time peopl locat equip event pinki safeti stuff bu organis pollut realli restaurantexcel dj approx sky recommend drove rush park distanc like sport rock experienceadventur huntawesom obt standard min paintbal cottag challe

# TF-IDF vectorization and cosine similarity

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [4]:
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

In [5]:
tfid_matrix = vectorizer.fit_transform(df['Review'])

In [9]:
vectorizer.get_feature_names()

D:\Anaconda3distribution\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aa',
 'aaa',
 'aaaaa',
 'aaaaaaaaaaaaaaaaaloc',
 'aaaaaaaaaaaaaaaand',
 'aaaaaaaaaaaaaahplac',
 'aaaaaaaaaamaz',
 'aaaaaaaaaamazinglov',
 'aaaaaaaaamaz',
 'aaaaaaaamaz',
 'aaaaaaaamazingcircular',
 'aaaaaaaand',
 'aaaaaaah',
 'aaaaaaahhhhhhh',
 'aaaaaaazaz',
 'aaaaaamaz',
 'aaaaamaz',
 'aaaaand',
 'aaaah',
 'aaaahbeach',
 'aaaahhhhh',
 'aaaamaz',
 'aaaamazingtempl',
 'aaaarceeeeeebeeeee',
 'aaaarthi',
 'aaaarti',
 'aaaawesometapesh',
 'aaadhi',
 'aaadi',
 'aaagcentr',
 'aaagumb',
 'aaah',
 'aaaha',
 'aaahhh',
 'aaahhhaaa',
 'aaam',
 'aaamaz',
 'aaaraku',
 'aaarathi',
 'aaarati',
 'aaarey',
 'aaarm',
 'aaarmi',
 'aaarthi',
 'aaarthivisit',
 'aaarti',
 'aaartimainli',
 'aaartiord',
 'aaartiyear',
 'aaaseen',
 'aaaya',
 'aab',
 'aabaad',
 'aabest',
 'aabha',
 'aabhathsakayewwarar',
 'aabhishakamknow',
 'aabi',
 'aabian',
 'aabideen',
 'aabig',
 'aabshar',
 'aabut',
 'aabyr',
 'aac',
 'aaccordinglybest',
 'aach',
 'aacha',
 'aachaar',
 'aachan',
 'aachar',
 'aachariyar',
 'aacharya',
 'a

In [10]:
tfid_matrix.shape

(15239, 921076)

In [11]:
tfid_matrix[0]

<1x921076 sparse matrix of type '<class 'numpy.float64'>'
	with 193 stored elements in Compressed Sparse Row format>

In [11]:
cosine_similarity1 = linear_kernel(tfid_matrix[0:5000, :], tfid_matrix) 

In [13]:
cosine_similarity2 = linear_kernel(tfid_matrix[5000:10000, :], tfid_matrix) 

In [14]:
cosine_similarity3 = linear_kernel(tfid_matrix[10000:15239, :], tfid_matrix) 

In [6]:
cosine_similarity = linear_kernel(tfid_matrix, tfid_matrix) 

In [9]:
cosine_similarity3.shape

(5239, 15239)

# Recommender Function

In [17]:
def recommend(City, Place):
    index = df[df['City'] == City][df['Place']==Place].index[0]
    if(index<5000):
        distances = sorted(list(enumerate(cosine_similarity1[index])), reverse=True,key = lambda x: x[1])
    elif(index<10000):
        distances = sorted(list(enumerate(cosine_similarity2[5000-index])), reverse=True,key = lambda x: x[1])
    else:
        distances = sorted(list(enumerate(cosine_similarity3[10000-index])), reverse=True,key = lambda x: x[1])        
    for i in distances[1:6]:
        print(df.iloc[i[0]].City,", ", df.iloc[i[0]].Place)

In [18]:
recommend('Agra', 'Taj Mahal')

Agra ,  Agra Fort
Amer ,  Amber Fort
New Delhi ,  Humayun's Tomb
New Delhi ,  Swaminarayan Akshardham
New Delhi ,  Qutub Minar


C:\Users\HP\AppData\Local\Temp\ipykernel_22556\2354172326.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = df[df['City'] == City][df['Place']==Place].index[0]


# Downloading the pickle file

In [8]:
import gzip, pickle, pickletools

In [9]:
pickle.dump(cosine_similarity, open("cosine_similarity.pkl", "wb"))

In [12]:
pickle.dump(cosine_similarity1, open("cosine_similarity1.pkl", "wb"))

In [15]:
pickle.dump(cosine_similarity2, open("cosine_similarity2.pkl", "wb"))

In [16]:
pickle.dump(cosine_similarity3, open("cosine_similarity3.pkl", "wb"))

In [10]:
pickle.dump(df, open("travel.pkl", "wb"))

In [20]:
def download_pickle(filepath, data):
    with gzip.open(filepath, "wb") as f:
        pickled = pickle.dumps(data)
        optimized_pickle = pickletools.optimize(pickled)
        f.write(optimized_pickle)

In [14]:
download_pickle("cosine_similarity1.pkl", cosine_similarity1)

In [15]:
download_pickle("cosine_similarity2.pkl", cosine_similarity2)

In [21]:
download_pickle("cosine_similarity3.pkl", cosine_similarity3)

In [22]:
download_pickle("travel.pkl", df)